# Optional - Advanced Solution 🧙🧙

The last solution is perfectly valid and we applied the rules from the lectures strictly. Now that we are more comfortable with Preprocessing with python, let's take a step back and see what we could have done differently by digging into the interpretation of the variables a little deeper.

1. Load the titanic dataset again

In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv('src/titanic.csv')
dataset.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

Let's explore the features more in details and try to extract more information than previously:

**A. Preprocessing to be planned with pandas**

**Unnecessary columns for prediction, to be thrown away** :
- _PassengerId_ and _Name_ are passenger identifiers, we won't use them for prediction (these columns don't contain any information)

:::tip Actually, _Name_ contains useful information !
As it is true that _Name_ cannot be used as such for prediction, it contains valuable information on the socio-economic background of the passenger in the form of their title. We will try and extract a _Title_ variable from the variable _Name_
:::

- _Ticket_ and _Cabin_ have too many different modalities, they might not be very useful and if we had to pass them in OneHotEncoding, they would make the number of columns explode in relation to the number of rows.

:::tip We can do something with the _Cabin_ variable !
_Ticket_ and _Cabin_ do have way too many modalities in order to be useful for prediction, however, the _Cabin_ variable can easily be used after a slight transformation : let's create a new variable _HasCabin_ which is equal to 1 when the passenger has a cabin number and 0 otherwise.
:::

**Columns with too many missing values, to be discarded** : Cabin


**Target variable/target (Y) that we will try to predict, to separate from the others** : Survived

**------------**

**B. Preprocessings to be planned with scikit-learn**.

**Explanatory variables (X)**
We need to identify which columns contain categorical variables and which columns contain numerical variables, as they will be treated differently.

- Categorical variables : Sex, Embarked, HasCabin, Title
- Numerical variables : Class, Age, Bbsp, Parch, Fare.

In this dataset, we have both types of variables. It will thus be necessary to plan to create a numeric_transformer (which will call the StandardScaler class) and a categorical_transformer (which will call the OneHotEncoder class). Moreover, as we observe missing values in the _Age_ and _Embarked_ columns, we will have to plan to call the SimpleImputer class to handle the missing values. 

**Target variable Y**
Here, the target variable Y is categorical (survival vs. death) but we notice that it is already encoded in numbers (1 vs. 0). It will therefore not be necessary to go through a label encoding step.

## Preprocessing - pandas part ##
2. Create a column _HasCabin_ in the dataset that is constant equal to 1

In [3]:
dataset["HasCabin"] = 1
dataset.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  HasCabin  
0      0         A/5 21171   7.2500   NaN        S         1  
1      0          PC 17599  71.2833   C85        C         1  
2      0  STON/O2. 3101282   7.9250   NaN        S         1  
3      0            113803  53.1000  C123        S         1  
4      0            373450   8.0500   NaN        S         1

3. Using a mask, change the value of the variable _HasCabin_ to 0 wherever Cabin is missing.

In [4]:
dataset.loc[dataset["Cabin"].isnull(),"HasCabin"] = 0
dataset.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  HasCabin  
0      0         A/5 21171   7.2500   NaN        S         0  
1      0          PC 17599  71.2833   C85        C         1  
2      0  STON/O2. 3101282   7.9250   NaN        S         0  
3      0            113803  53.1000  C123        S         1  
4      0            373450   8.0500   NaN        S         0

5. Create a column _Title_ that only contains the title extracted from the _Name_ variable. 
:::tip Remember pandas handles columns of strings efficiently
Some method from [the str module](https://docs.python.org/3.3/library/stdtypes.html?highlight=split) can be helpful 😉
You can create a function that allows to extract the title from one element of the column, and then use the `apply()` method to apply this function to the whole column.
:::

In [5]:
def get_title(name):
    return name.split(', ')[1].split('.')[0]

In [6]:
# Create a new feature Title, containing the titles of passenger names
dataset['Title'] = dataset['Name'].apply(get_title)
dataset.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  HasCabin Title  
0      0         A/5 21171   7.2500   NaN        S         0    Mr  
1      0          PC 17599  71.2833   C85        C         1   Mrs  
2      0  STON/O2. 3101282   7.9250   NaN        S         0  Miss  
3      0            113803  53.1000  C123        S         1   Mrs  
4      0            373450   8.0500   NaN        S         0    Mr

6. Display all the possible values and number of instances of each of these values in your dataset for the new _Title_ variable.

In [7]:
dataset["Title"].value_counts()

Mr              517
Miss            182
Mrs             125
Master           40
Dr                7
Rev               6
Mlle              2
Major             2
Col               2
the Countess      1
Capt              1
Ms                1
Sir               1
Lady              1
Mme               1
Don               1
Jonkheer          1
Name: Title, dtype: int64

7. Some of these values represent only very few instances, and other values seem to represent the similar categories of people. Bring the similar categories under one name, and create a new category called _Rare_ that will represent all the underrepresented modalities.

In [8]:
# Group all synonimous titles under one category
dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

# Group all non-common titles into one single grouping "Rare"
dataset['Title'] = dataset['Title'].replace(['Lady', 'the Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

dataset['Title'].value_counts()

Mr        517
Miss      185
Mrs       126
Master     40
Rare       23
Name: Title, dtype: int64

8. Now that we are done squeezing some extra information out of our variables, let's reproduce all the subsequent steps from the first solution and let's compare our models' performances.

In [9]:
# Drop useless columns / columns with too many missing values
useless_cols = ['PassengerId', 'Name', 'Ticket', 'Cabin']

print("Dropping useless columns...")
dataset = dataset.drop(useless_cols, axis=1)
print("...Done.")
dataset.head()

Dropping useless columns...
...Done.


Survived  Pclass     Sex   Age  SibSp  Parch     Fare Embarked  HasCabin  \
0         0       3    male  22.0      1      0   7.2500        S         0   
1         1       1  female  38.0      1      0  71.2833        C         1   
2         1       3  female  26.0      0      0   7.9250        S         0   
3         1       1  female  35.0      1      0  53.1000        S         1   
4         0       3    male  35.0      0      0   8.0500        S         0   

  Title  
0    Mr  
1   Mrs  
2  Miss  
3   Mrs  
4    Mr

In [10]:
# Separate target variable Y from features X
target_name = 'Survived'

print("Separating labels from features...")
Y = dataset.loc[:,target_name]
X = dataset.drop(target_name, axis = 1) # Keeping all columns
print("...Done.")
print(Y.head())
print()
print(X.head())

Separating labels from features...
...Done.
0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

   Pclass     Sex   Age  SibSp  Parch     Fare Embarked  HasCabin Title
0       3    male  22.0      1      0   7.2500        S         0    Mr
1       1  female  38.0      1      0  71.2833        C         1   Mrs
2       3  female  26.0      0      0   7.9250        S         0  Miss
3       1  female  35.0      1      0  53.1000        S         1   Mrs
4       3    male  35.0      0      0   8.0500        S         0    Mr


In [11]:
from sklearn.model_selection import train_test_split
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=0)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



In [12]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [13]:
# Create pipeline for numeric features
numeric_features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'] # Names of numeric columns in X_train/X_test
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')), # missing values in Age will be replaced by columns' mean
    ('scaler', StandardScaler())
])

In [14]:
from sklearn.preprocessing import OneHotEncoder

In [15]:
# Create pipeline for categorical features
categorical_features = ['Sex', 'Embarked', 'HasCabin', 'Title'] # Names of categorical columns in X_train/X_test
categorical_transformer = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # missing values will be replaced by most frequent value
    ('encoder', OneHotEncoder(drop='first')) # first column will be dropped to avoid creating correlations between features
    ])

In [16]:
from sklearn.compose import ColumnTransformer

In [17]:
# Use ColumnTranformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Preprocessings on train set
print("Performing preprocessings on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print('...Done.')
print(X_train[0:5,:])
print()

# Preprocessings on test set
print("Performing preprocessings on test set...")
print(X_test.head())
X_test = preprocessor.transform(X_test) # Don't fit again !!
print('...Done.')
print(X_test[0:5,:])
print()

Performing preprocessings on train set...
     Pclass     Sex   Age  SibSp  Parch    Fare Embarked  HasCabin Title
545       1    male  64.0      0      0  26.000        S         0    Mr
37        3    male  21.0      0      0   8.050        S         0    Mr
214       3    male   NaN      1      0   7.750        Q         0    Mr
40        3  female  40.0      1      0   9.475        S         0   Mrs
236       2    male  44.0      1      0  26.000        S         0    Mr
...Done.
[[-1.60067161e+00  2.61131471e+00 -4.63468368e-01 -4.65997851e-01
  -1.09604554e-01  1.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 8.10688409e-01 -6.78358906e-01 -4.63468368e-01 -4.65997851e-01
  -4.71133941e-01  1.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 8.10688409e-01 -2.71796941e-16  4.31545801e-01 -4.65997851e-01
  -4.771762

### Training model

In [18]:
from sklearn.linear_model import LogisticRegression

In [19]:
# Train model
model = LogisticRegression()

print("Training model...")
model.fit(X_train, Y_train) # Training is always done on train set !!
print("...Done.")

Training model...
...Done.


### Predictions

In [20]:
# Predictions on training set
print("Predictions on training set...")
Y_train_pred = model.predict(X_train)
print("...Done.")
print(Y_train_pred[0:5])
print()

Predictions on training set...
...Done.
[0 0 0 1 0]



In [21]:
# Predictions on test set
print("Predictions on test set...")
Y_test_pred = model.predict(X_test)
print("...Done.")
print(Y_test_pred[0:5])
print()

Predictions on test set...
...Done.
[0 0 0 1 1]



### Performances evaluation

In [22]:
from sklearn.metrics import accuracy_score

In [23]:
# Print scores
print("Accuracy on training set : ", accuracy_score(Y_train, Y_train_pred))
print("Accuracy on test set : ", accuracy_score(Y_test, Y_test_pred))

Accuracy on training set :  0.8282694848084544
Accuracy on test set :  0.8208955223880597


Tada 🥳 If you worked well, the score has improved a bit!
This example shows that by adding a little additional information to a model, it is possible to create a significant impact on the performances of the predictive model. Knowing and applying the preprocessing guidelines is great but always remember to check for two important things before you proceed :

* If a variable contains missing values, ask yourself why this value is missing and whether you could use it as information to feed the model with. In the above example, the fact that a passenger does not have a cabin number simply means that they have no cabin. It is very common for missing values to contain hidden meaning, completely random missing values (caused by a bug or other unpredictable causes) are very rare.

* When a non-numerical variable is not usable as is, always ask yourself whether you could still extract some information from it. Here the _Name_ variable cannot be used, however it mentions the passenger's title which can be useful information.